## MACS30150 PS8 (Question 2)
### Dr. Richard Evans
### Submitted by Junho Choi

Let us import the necessary functions and packages.

In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.ensemble import RandomForestRegressor as RanForReg
from sklearn.ensemble import RandomForestClassifier as RanForCla
from sklearn.linear_model import LogisticRegression as LR
from sklearn.svm import SVC
from sklearn.model_selection import LeaveOneOut, KFold
from sklearn.metrics import classification_report

## Problem 2

Before proceeding with the analyses in the sub-problems, let us read in the data. Before a previous problem set, we know that there exists missing data for displacement written as `?`. Therefore, we set the `na_values` as such and drop the rows with missing data.

In [3]:
auto = pd.read_csv('Auto.csv', na_values='?')
print(auto.shape)

(397, 9)


In [4]:
auto = auto.dropna(axis=0)
print(auto.shape) ## after dropping the NA values

(392, 9)


Let us create the variables `mpg_high`, `orgn1`, and `orgn2` as directed by the question.

In [5]:
med = auto['mpg'].median()
auto['mpg_high'] = 0
auto.loc[auto['mpg'] >= med, 'mpg_high'] = 1

In [6]:
auto['orgn1'] = 0
auto['orgn2'] = 0
auto.loc[auto['origin'] == 1, 'orgn1'] = 1
auto.loc[auto['origin'] == 2, 'orgn2'] = 1

The resulting dataset would look something like this.

In [7]:
auto.head(8)

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,mpg_high,orgn1,orgn2
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,0,1,0
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,0,1,0
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,0,1,0
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,0,1,0
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,0,1,0
5,15.0,8,429.0,198.0,4341,10.0,70,1,ford galaxie 500,0,1,0
6,14.0,8,454.0,220.0,4354,9.0,70,1,chevrolet impala,0,1,0
7,14.0,8,440.0,215.0,4312,8.5,70,1,plymouth fury iii,0,1,0


Below code checks whether the two variables `orgn1` and `orgn2` have been successfully created.

In [8]:
## checking whether orgn1 and orgn2 variables have been created well
print(auto.groupby(['origin']).size())
print()
print(auto.orgn1.sum(), auto.orgn2.sum())

origin
1    245
2     68
3     79
dtype: int64

245 68


Let us set the y- and x-variables (i.e. the dependent variable and the regressors) to be used in the sub-problems.

In [9]:
y_auto = auto['mpg_high']
X_columns = ['cylinders', 'displacement', 'horsepower', 'weight', 
             'acceleration', 'year', 'orgn1', 'orgn2']
X_auto = auto[X_columns]

X_auto_vals = X_auto.values
y_auto_vals = y_auto.values

### Problem 2-(a)

For this part, I use the (revised) function `cv_kf_mlogit` that I implemented from a previous problem set. It is as follows.

In [10]:
def cv_kf_mlogit(Xvals, yvals, k=4, rtn_vectors=True, random=25):
    ## creating the splits
    kf = KFold(n_splits=k, shuffle=True, random_state=random)
    kf.get_n_splits(Xvals)

    ## creating the vector of KFolds MSE
    MSE_vec_kf = np.zeros(k)
    k_ind = int(0)
    
    for train_idx, test_idx in kf.split(Xvals):
        ## splitting training and test sets
        X_tr_kf, X_test_kf = Xvals[train_idx], Xvals[test_idx]
        y_tr_kf, y_test_kf = yvals[train_idx], yvals[test_idx]

        ## making the prediction
        mlogit_kf = LR(random_state=random, solver='lbfgs',
                       multi_class='multinomial').fit(X_tr_kf, y_tr_kf)
        y_pred_kf = mlogit_kf.predict(X_test_kf)

        ## indicator function for categorical variables
        MSE_indicator_kf = (y_pred_kf != y_test_kf).mean()

        ## MSE vector for the specific k-fold
        MSE_vec_kf[k_ind] = MSE_indicator_kf
        print('Fold #{}:'.format(k_ind+1), 'MSE is {}'.format(MSE_indicator_kf))

        ## creating the vectors of actual and test yvals
        if k_ind == 0:
            y_act, y_pred = y_test_kf, y_pred_kf

        else:
            y_act = np.hstack((y_act, y_test_kf))
            y_pred = np.hstack((y_pred, y_pred_kf))

        k_ind += 1

    cv_kf = MSE_vec_kf.mean()
    print('-------------------------------')
    print('CV_KF is {}'.format(cv_kf))

    if rtn_vectors:
        return cv_kf, y_act, y_pred

    else:
        return cv_kf

Using the said function, the MSE of the K-folds (with $k=4$) crossvalidation can be found as approximately $0.0969$.

In [11]:
cv_kf, y_act, y_pred = \
    cv_kf_mlogit(X_auto_vals, y_auto_vals, k=4,
                 rtn_vectors=True, random=25)

Fold #1: MSE is 0.14285714285714285
Fold #2: MSE is 0.09183673469387756
Fold #3: MSE is 0.07142857142857142
Fold #4: MSE is 0.08163265306122448
-------------------------------
CV_KF is 0.09693877551020408


C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In addition, the error rates (that is , $1-\text{precision}$) for each group (i.e. groups `mpg_high=0` and `mpg_high=1`) are as follows. For the low-mpg group (i.e. `mpg_high=1`), the error rate is approximately $0.08$ ($=1-0.92$). On the other hand, for the high-mpg group (i.e. `mpg_high=0`), the error rate is approximately $0.11$ ($=1-0.89$).

In [12]:
print(classification_report(y_act, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.88      0.90       196
           1       0.89      0.92      0.90       196

   micro avg       0.90      0.90      0.90       392
   macro avg       0.90      0.90      0.90       392
weighted avg       0.90      0.90      0.90       392



### Problem 2-(b)

Once again, because the random forest classifier also relies on randomness, let us set `random_state=25`. Using the below chunks of code, let us conduct hyperparameter tuning for random forest classifier. 

In [13]:
rfc_gen = RanForCla(random_state=25)

param_dist3 = {
    'n_estimators': [10, 200],
    'max_depth': [3, 8],
    'min_samples_split': sp_randint(2, 20),
    'min_samples_leaf': sp_randint(2, 20),
    'max_features': sp_randint(1, 8)
}

In [14]:
random_search3 = \
    RandomizedSearchCV(rfc_gen, param_distributions=param_dist3,
                       n_iter=100, n_jobs=-1, cv=4, random_state=25,
                       scoring='neg_mean_squared_error')

In [18]:
random_search3.fit(X_auto_vals, y_auto_vals)
print('RandBestEstimator3=', random_search3.best_estimator_)
print('RandBestParams3=', random_search3.best_params_)
print('RandBestScore3=', -random_search3.best_score_)

RandBestEstimator3= RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=8, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=15, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=25, verbose=0, warm_start=False)
RandBestParams3= {'max_depth': 8, 'max_features': 3, 'min_samples_leaf': 15, 'min_samples_split': 2, 'n_estimators': 10}
RandBestScore3= 0.08928571428571429


It can then be seen from above that the optimal tuning parameter values are as follows: `max_depth` of 8, `max_features` of 3, `min_samples_leaf` of 15, `min_samples_split` of 2, and finally `n_estimators` of 10. The MSE of optimal results is found to be approximately $0.0893$.

### Problem 2-(c)

Let us tune the hyperparameters for the support vector classifier (`sklearn.svm.SVC`) using the following chunks of code. As the question directs, I have initialized the kernel to be radial basis function (`kernel=rbf`).

In [22]:
svc = SVC(kernel='rbf')

param_dist4 = {
    'C': sp_uniform(loc=0.2, scale=4.0),
    'gamma': ['scale', 'auto'],
    'shrinking': [True, False]
}

In [23]:
random_search4 = \
   RandomizedSearchCV(svc, param_distributions=param_dist4,
                      n_iter=100, n_jobs=-1, cv=4, random_state=25,
                      scoring='neg_mean_squared_error')

In [21]:
random_search4.fit(X_auto_vals, y_auto_vals)
print('RandBestEstimator4=', random_search4.best_estimator_)
print('RandBestParams4=', random_search4.best_params_)
print('RandBestScore4=', -random_search4.best_score_)

RandBestEstimator4= SVC(C=1.8094629152568114, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=False,
  tol=0.001, verbose=False)
RandBestParams4= {'C': 1.8094629152568114, 'gamma': 'scale', 'shrinking': False}
RandBestScore4= 0.11479591836734694


As seen from the above result, the optimized values for the parameters `C` (penalty parameter), `gamma` (kernel coefficient), and `shrinking` are as follows: (approximately) $1.8095$, `scale`, and `False` respectively. The MSE of the optimal results is approximately $0.1148$.

### Problem 2-(d)

From the above parts 2-(a) to 2-(c), the model in 2-(b) -- that is, random forest classifier with maximum depth of 8, maximum number of features of 3, minimum samples in leaf of 15, minimum samples split of 2, and number of estimators being 10 -- had the lowest value of MSE among the three models. Therefore, I would say the model in 2-(b) is the best predictor of `mpg_high` (based on MSE).